In [ ]:
import cv2
import numpy as np
import cv2.aruco as aruco

cap = cv2.VideoCapture('/dev/video2')

aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)
parameters = aruco.DetectorParameters()

data = np.load('/home/choigh/WS/Test_Tools_code/calib_data.npz')
camera_matrix = data['mtx']
dist_coeffs = data['dist']
marker_length = 0.03

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    corners, ids, rejected = aruco.detectMarkers(gray, aruco_dict, parameters=parameters)

    if ids is not None:
        rvecs, tvecs, _ = aruco.estimatePoseSingleMarkers(corners, 0.03, camera_matrix, dist_coeffs)

        for i in range(len(ids)):
            aruco.drawDetectedMarkers(frame, corners, ids)
            cv2.drawFrameAxes(frame, camera_matrix, dist_coeffs, rvecs[i], tvecs[i], 0.03)

            # 거리 정보 출력
            z = tvecs[i][0][2]
            print(f"ID {ids[i][0]} | X={tvecs[i][0][0]:.3f}  Y={tvecs[i][0][1]:.3f}  Z={z:.3f}")

            # 화면 표시용 텍스트
            cX, cY = int(corners[i][0][0][0]), int(corners[i][0][0][1])
            cv2.putText(frame, f"ID:{ids[i][0]} Z={z:.2f}m", (cX, cY - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    cv2.imshow("Aruco Detection", frame)
    if cv2.waitKey(1) & 0xFF == 27:  # ESC to exit
        break

cap.release()
cv2.destroyAllWindows()

In [9]:
import numpy as np

tvec_history = []

for _ in range(100):
    
    corners, ids, rejected = aruco.detectMarkers(gray, aruco_dict, parameters=parameters)
    rvecs, tvecs, _ = aruco.estimatePoseSingleMarkers(corners, 0.03, camera_matrix, dist_coeffs)

    if ids is not None:
        tvec_history.append(tvecs[0][0])  # 첫 번째 마커만 추적

tvec_history = np.array(tvec_history)
mean = np.mean(tvec_history, axis=0)
std = np.std(tvec_history, axis=0)
print(f"평균 좌표: {mean}")
print(f"표준편차(오차): {std}")


평균 좌표: [-0.02101626 -0.01527056  0.21711994]
표준편차(오차): [2.42861287e-17 1.04083409e-17 1.94289029e-16]
